# Validation - distribution of GT and optimized AP potentials/currents

This notebook evaluates qualitatively the distributions of membrane potentials and transmembrane currents of an action potential over the neuron morphology.

In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import sys
import shutil

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import matplotlib.pyplot as plt
from scipy.spatial import distance
import MEAutility as mu
import json
import time
import numpy as np
from pathlib import Path

import multimodalfitting as mf

%matplotlib notebook

In [ ]:
probe_type = "planar"
model_name = "hay_ais" # "hay", "hay_ais", "hay_ais_hillock"
cell_models_folder = Path("..") / "cell_models"

model_folder = cell_models_folder / f"{model_name}"

In [ ]:
result_folder = Path("..") / "results" / '211124' 
pkl_file_name = "runs.pkl"

data = pickle.load(open(result_folder / pkl_file_name, 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")

In [ ]:
opt_soma = df_model.query("feature_set == 'soma'")
opt_extra = df_model.query("feature_set == 'extra'")
print(f"Somatic optimizations: {len(opt_soma)}")
print(f"Extra optimizations: {len(opt_extra)}")

In [ ]:
protocols_file = model_folder / "fitting" / "efeatures" / "protocols_BPO_all.json"
features_file = model_folder / "fitting" / "efeatures" / "features_BPO_all.json"

In [ ]:
cell = mf.create_ground_truth_model(model_name=model_name, release=False)
cell_release = mf.create_ground_truth_model(model_name=model_name, release=True)

probe = mf.define_electrode(probe_type=probe_type)

param_names = [param.name for param in cell.params.values() if not param.frozen]
# sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe, mechs_folders=model_folder)

params_release = {}
for param in cell_release.params_by_names(param_names):
    params_release[param.name] = param.value

In [ ]:
protocol_for_eap = "IDrest_300"

### Define more recording points 

In [ ]:
positions = np.array([[-62, 828], [-27, 546], [-27, 85], [14, -26], [28, -290]])
position_names = ["apical_distal", "apical_middle", "apical_proximal", "ais_middle", "axon"]

In [ ]:
extra_kwargs = dict(fs=20,
                    fcut=[300, 6000],
                    filt_type="filtfilt",
                    ms_cut=[3, 10])

In [ ]:
eva_extra = mf.create_evaluator(
    model_name=model_name,
    feature_set="extra",
    extra_strategy="all",
    protocols_with_lfp=protocol_for_eap,
    **extra_kwargs
)

In [ ]:
extra_recordings = mf.utils.extra_recordings_from_positions(cell_release, eva_extra.sim, positions, position_names)

In [ ]:
eva_extra = mf.create_evaluator(
    model_name=model_name,
    feature_set="extra",
    extra_strategy="all",
    protocols_with_lfp=protocol_for_eap,
    extra_recordings=dict(IDrest_300=extra_recordings), 
    **extra_kwargs
)

In [ ]:
eva_extra.fitness_protocols["IDrest_300"].recordings

In [ ]:
len(extra_recordings)

# Load protocols and original features

In [ ]:
ais_recording = mf.utils.get_ais_extra_recordings()

In [ ]:
ais_recording

In [ ]:
# eva_extra = mf.create_evaluator(
#     model_name=model_name,
#     feature_set="extra",
#     extra_strategy="all",
#     protocols_with_lfp=protocol_for_eap,
#     extra_recordings=dict(IDrest_300=ais_recording), 
#     **extra_kwargs
# )

In [ ]:
# only run IDrest 300

In [ ]:
idrest = eva_extra.fitness_protocols["IDrest_300"]

In [ ]:
responses_release = eva_extra.run_protocol(idrest, param_values=params_release)

In [ ]:
mf.plot_responses(responses_release)